In [1]:
import requests as rq
import os
import re
import requests
import glob
import shutil
import csv
from bs4 import BeautifulSoup
import datetime
import numpy as np
import io
import timeit
import pandas as pd
from six.moves import urllib

In [2]:
# combine all the files in the folder to a pandas dataframe
cwd = os.getcwd()
path = os.path.join(cwd,'病媒蚊密度調查')
FileList = os.listdir(path)
dens = pd.DataFrame()
for file in FileList:
    df = pd.read_csv(os.path.join(path, file))
    df = df.dropna(subset = ['區別'])
    if not dens.empty:
        dens = pd.concat([dens,df])
    else:
        dens = df
dens.head(5)

C:\Users\yt335\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


,區別,台南市,容器指數,容器級數,布氏指數,布氏級數,日期,經度,緯度,臺南市,...,調查戶數,調查種類,調種類,里別,陽性容器合計,陽性容器戶內,陽性容器戶外,陽性容器數戶內,陽性容器數戶外,陽性戶數
0,東 區,臺南市,29,7,12.0,3.0,106/6/12,120.227472,22.9809,NaN,...,65.0,地區,NaN,崇信里,8.0,0.0,8.0,NaN,NaN,6.0
1,東 區,臺南市,41,9,15.0,3.0,106/6/12,120.221701,22.9835,NaN,...,60.0,地區,NaN,龍山里,9.0,0.0,9.0,NaN,NaN,5.0
2,東 區,臺南市,20,5,8.0,2.0,106/6/13,120.231800,22.9867,NaN,...,50.0,住宅,NaN,富強里,4.0,0.0,4.0,NaN,NaN,3.0
3,東 區,臺南市,17,5,4.0,1.0,106/6/13,120.244782,22.984,NaN,...,50.0,住宅,NaN,文聖里,2.0,0.0,2.0,NaN,NaN,2.0
4,東 區,臺南市,27,6,14.0,3.0,106/6/14,120.246946,22.9802,NaN,...,50.0,住宅,NaN,裕聖里,7.0,1.0,6.0,NaN,NaN,6.0


In [3]:
dens.shape

(46224, 25)

In [4]:
dens = dens.reset_index()

nogood = dens[~dens['日期'].str.contains('/')].index

for i in nogood:
    dens.loc[i,'日期'] = dens.loc[i-1,'日期']

In [5]:
dens.replace('南　區','南區', inplace=True)
dens.replace('東　區','東區', inplace=True)
dens.replace('北　區','北區', inplace=True)

In [6]:
dens[['Year','Month','Day']] = dens.日期.str.split('/',expand=True)
dens['Year'] = pd.to_numeric(dens['Year'])
dens.loc[dens.Year<1911,'Year'] = dens.loc[dens.Year<1911,'Year']+1911
dens['Year'] = dens['Year'].astype(str)
dens['Month'] = dens['Month'].astype(str)
dens['Day'] = dens['Day'].astype(str)
dens['Date'] = dens[['Year', 'Month', 'Day']].agg('/'.join, axis=1)
dens['Date'] = pd.to_datetime(dens['Date'])
dens['Week'] = dens['Date'].dt.strftime('%U')

In [7]:
dens.drop(columns = ['index','台南市','臺南市','調查容器合計','陽性容器合計','日期'], inplace=True)
dens.head(5)

,區別,容器指數,容器級數,布氏指數,布氏級數,經度,緯度,調查容器戶內,調查容器戶外,調查容器數戶內,...,陽性容器戶內,陽性容器戶外,陽性容器數戶內,陽性容器數戶外,陽性戶數,Year,Month,Day,Date,Week
0,東區,29,7,12.0,3.0,120.227472,22.9809,6.0,22.0,NaN,...,0.0,8.0,NaN,NaN,6.0,2017,6,12,2017-06-12,24
1,東區,41,9,15.0,3.0,120.221701,22.9835,3.0,19.0,NaN,...,0.0,9.0,NaN,NaN,5.0,2017,6,12,2017-06-12,24
2,東區,20,5,8.0,2.0,120.231800,22.9867,9.0,11.0,NaN,...,0.0,4.0,NaN,NaN,3.0,2017,6,13,2017-06-13,24
3,東區,17,5,4.0,1.0,120.244782,22.984,3.0,9.0,NaN,...,0.0,2.0,NaN,NaN,2.0,2017,6,13,2017-06-13,24
4,東區,27,6,14.0,3.0,120.246946,22.9802,5.0,21.0,NaN,...,1.0,6.0,NaN,NaN,6.0,2017,6,14,2017-06-14,24


In [8]:
dens_temp1 = dens[(dens['調查容器數戶內'].isnull())&(dens['調種類'].isnull())]
dens_temp1 = dens_temp1.drop(columns=['調種類','調查容器數戶內','調查容器數戶外','陽性容器數戶內','陽性容器數戶外'])

In [9]:
dens_temp2 = dens[(dens['調種類'].notnull())]
dens_temp2['調查種類'] = dens_temp2['調種類']
dens_temp2 = dens_temp2.drop(columns=['調種類','調查容器數戶內','調查容器數戶外','陽性容器數戶內','陽性容器數戶外'])

C:\Users\yt335\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
dens_temp3 = dens[dens['調查容器數戶內'].notnull()]
dens_temp3['調查容器合計'] = dens_temp3['調查容器數戶內'] + dens_temp3['調查容器數戶外']
dens_temp3['陽性容器合計'] = dens_temp3['陽性容器數戶內'] + dens_temp3['陽性容器數戶外']
dens_temp3['調查容器戶內'] = dens_temp3['調查容器數戶內']
dens_temp3['陽性容器戶內'] = dens_temp3['陽性容器數戶內']
dens_temp3['調查容器戶外'] = dens_temp3['調查容器數戶外']
dens_temp3['陽性容器戶外'] = dens_temp3['陽性容器數戶外']
dens_temp3 = dens_temp3.drop(columns=['調種類','調查容器數戶內','調查容器數戶外','陽性容器數戶內','陽性容器數戶外'])

C:\Users\yt335\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\yt335\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\yt335\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is tryin

In [11]:
dens_clean = pd.concat([dens_temp1,dens_temp2,dens_temp3])
dens_clean

C:\Users\yt335\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Date,Day,Month,Week,Year,區別,容器指數,容器級數,布氏指數,布氏級數,...,調查容器合計,調查容器戶內,調查容器戶外,調查戶數,調查種類,里別,陽性容器合計,陽性容器戶內,陽性容器戶外,陽性戶數
0,2017-06-12,12,6,24,2017,東區,29,7,12.0,3.0,...,NaN,6.0,22.0,65.0,地區,崇信里,NaN,0.0,8.0,6.0
1,2017-06-12,12,6,24,2017,東區,41,9,15.0,3.0,...,NaN,3.0,19.0,60.0,地區,龍山里,NaN,0.0,9.0,5.0
2,2017-06-13,13,6,24,2017,東區,20,5,8.0,2.0,...,NaN,9.0,11.0,50.0,住宅,富強里,NaN,0.0,4.0,3.0
3,2017-06-13,13,6,24,2017,東區,17,5,4.0,1.0,...,NaN,3.0,9.0,50.0,住宅,文聖里,NaN,0.0,2.0,2.0
4,2017-06-14,14,6,24,2017,東區,27,6,14.0,3.0,...,NaN,5.0,21.0,50.0,住宅,裕聖里,NaN,1.0,6.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9742,2015-09-29,29,9,39,2015,左鎮區,4,2,2.0,1.0,...,28.0,1.0,27.0,50.0,住宅,澄山里,1.0,0.0,1.0,1.0
9743,2015-09-03,3,9,35,2015,左鎮區,5,2,4.0,1.0,...,42.0,2.0,40.0,50.0,住宅,光和里,2.0,0.0,2.0,1.0
9744,2015-09-30,30,9,39,2015,左鎮區,5,2,2.0,1.0,...,21.0,0.0,21.0,50.0,住宅,內庄里,1.0,0.0,1.0,1.0
9745,2015-09-30,30,9,39,2015,左鎮區,4,2,2.0,1.0,...,28.0,0.0,28.0,50.0,住宅,睦光里,1.0,0.0,1.0,1.0


In [12]:
dens_clean['調查容器合計'] = dens_clean['調查容器戶內'] + dens_clean['調查容器戶外']
dens_clean['陽性容器合計'] = dens_clean['陽性容器戶內'] + dens_clean['陽性容器戶外']
dens_clean = dens_clean.replace('查無經緯度', np.nan)
dens_clean['經度'].fillna(value=0, inplace=True)
dens_clean['緯度'].fillna(value=0, inplace=True)
dens_clean['緯度'] = dens_clean['緯度'].astype(float)
dens_clean['調查戶數'] = dens_clean['調查戶數'].astype(float)
dens_clean['陽性戶數'] = dens_clean['陽性戶數'].astype(float)
dens_clean['調查容器合計'] = dens_clean['調查容器合計'].astype(float)
dens_clean['陽性容器合計'] = dens_clean['陽性容器合計'].astype(float)

In [13]:
target_area = list(dens_clean.groupby(['區別','里別']).count().sort_values('Date').index[dens_clean.groupby(['區別','里別']).count().sort_values('Date').Date>=150])
target_area

[('永康區', '北灣里'),
 ('東區', '龍山里'),
 ('永康區', '大灣里'),
 ('東區', '小東里'),
 ('安南區', '安富里'),
 ('東區', '泉南里'),
 ('永康區', '大橋里'),
 ('北區', '大豐里'),
 ('北區', '和順里'),
 ('南區', '大林里'),
 ('東區', '崇明里'),
 ('南區', '國宅里'),
 ('仁德區', '土庫里'),
 ('東區', '後甲里'),
 ('仁德區', '大甲里'),
 ('中西區', '光賢里'),
 ('南區', '明興里'),
 ('永康區', '崑山里'),
 ('南區', '文南里'),
 ('東區', '德高里'),
 ('永康區', '王行里'),
 ('仁德區', '上崙里'),
 ('南區', '新昌里'),
 ('中西區', '開山里'),
 ('中西區', '西湖里'),
 ('中西區', '大涼里'),
 ('仁德區', '太子里'),
 ('東區', '和平里'),
 ('北區', '力行里'),
 ('中西區', '郡王里'),
 ('永康區', '復國里'),
 ('永康區', '埔園里'),
 ('北區', '成德里'),
 ('東區', '裕聖里'),
 ('永康區', '復華里'),
 ('中西區', '永華里'),
 ('仁德區', '仁義里'),
 ('永康區', '甲頂里'),
 ('北區', '永祥里'),
 ('仁德區', '保安里'),
 ('永康區', '二王里'),
 ('中西區', '藥王里'),
 ('南區', '金華里'),
 ('北區', '正覺里'),
 ('中西區', '西賢里'),
 ('永康區', '永康里'),
 ('永康區', '勝利里'),
 ('北區', '中樓里'),
 ('仁德區', '後壁里'),
 ('東區', '大同里'),
 ('東區', '大學里'),
 ('東區', '成大里'),
 ('安平區', '怡平里'),
 ('安平區', '漁光里'),
 ('安平區', '華平里'),
 ('安平區', '育平里'),
 ('中西區', '赤嵌里'),
 ('安平區', '平安里'),
 ('仁德區', '成功里'),
 ('永康區', '五王里'),
 ('安

In [18]:
dens_target = pd.DataFrame()
chi = []
li = []
longti = []
alti = []

for i in range(len(target_area)):

    df = dens_clean[(dens_clean['區別']==target_area[i][0]) & (dens_clean['里別']==target_area[i][1])]
    longtitude = []
    altitude = []
    for j in range(len(df)):
        if df.iloc[j].經度>100:
            longtitude.append(df.iloc[j].經度)
        if df.iloc[j].緯度 != 0:
            if df.iloc[j].緯度 < 80:
                altitude.append(df.iloc[j].緯度)
  
    tar_lon = np.mean(longtitude)   
    tar_al = np.mean(altitude)
  
    chi.append(target_area[i][0])
    li.append(target_area[i][1])
    longti.append(tar_lon)
    alti.append(tar_al)
    
    if not dens_target.empty:
        dens_target = pd.concat([dens_target,df])
    else:
        dens_target = df
dens_target

,Date,Day,Month,Week,Year,區別,容器指數,容器級數,布氏指數,布氏級數,...,調查容器合計,調查容器戶內,調查容器戶外,調查戶數,調查種類,里別,陽性容器合計,陽性容器戶內,陽性容器戶外,陽性戶數
229,2017-07-05,5,7,27,2017,永康區,6,3,4.0,1.0,...,35.0,2.0,33.0,50.0,住宅,北灣里,2.0,0.0,2.0,2.0
347,2017-07-12,12,7,28,2017,永康區,0,0,0.0,0.0,...,5.0,2.0,3.0,50.0,住宅,北灣里,0.0,0.0,0.0,0.0
803,2017-08-10,10,8,32,2017,永康區,9,3,2.0,1.0,...,11.0,1.0,10.0,50.0,住宅,北灣里,1.0,0.0,1.0,1.0
805,2017-08-10,10,8,32,2017,永康區,0,0,0.0,0.0,...,23.0,3.0,20.0,50.0,住宅,北灣里,0.0,0.0,0.0,1.0
948,2017-08-16,16,8,33,2017,永康區,9,3,2.0,1.0,...,11.0,3.0,8.0,50.0,住宅,北灣里,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3902,2015-07-16,16,7,28,2015,安平區,13,4,4.0,1.0,...,15.0,0.0,15.0,50.0,住宅,億載里,2.0,0.0,2.0,2.0
3967,2015-09-01,1,9,35,2015,安平區,67,9,24.0,4.0,...,18.0,0.0,18.0,50.0,住宅,億載里,12.0,0.0,12.0,7.0
3974,2015-09-02,2,9,35,2015,安平區,11,4,4.0,1.0,...,19.0,0.0,19.0,50.0,住宅,億載里,2.0,0.0,2.0,0.0
3989,2015-09-04,4,9,35,2015,安平區,6,3,5.0,2.0,...,16.0,7.0,9.0,22.0,住宅,億載里,1.0,0.0,1.0,1.0


In [19]:
dens_target['陽性戶數'].fillna(value=0, inplace=True)

location = pd.DataFrame({'區別':chi,'里別':li,'經度':longti,'緯度':alti})

dens_target = dens_target.drop(columns = ['經度','緯度'])
dens_target = pd.merge(dens_target,location)
dens_target = dens_target.drop(columns = ['調查種類','調查容器戶內','陽性容器戶內','調查容器戶外','陽性容器戶外','布氏級數','容器級數','布氏指數','容器指數'])
dens_target = dens_target.reindex(columns = ['Date','Year','Month','Day','Week','區別','里別','調查戶數','陽性戶數','調查容器合計','陽性容器合計','經度','緯度'])

dens_target_fill = pd.DataFrame()

In [20]:
for i in range(len(target_area)):

    df = dens_target[(dens_target['區別']==target_area[i][0]) & (dens_target['里別']==target_area[i][1])]
    duplicate_date = df[df['Date'].duplicated() == True].Date.unique()

    eliminate = []

    for i in duplicate_date:

        combine = []
        temp = df[df['Date'] == i]
        combine.extend([temp.iloc[0].Date,temp.iloc[0].Year,temp.iloc[0].Month,temp.iloc[0].Day,temp.iloc[0].Week,temp.iloc[0].區別,temp.iloc[0].里別])
        inv = 0
        sun = 0
        invb = 0
        sunb = 0

        for i in range(len(temp)):
            inv += temp.iloc[0].調查戶數
            sun += temp.iloc[0].陽性戶數
            invb += temp.iloc[0].調查容器合計
            sunb += temp.iloc[0].陽性容器合計

        combine.extend([inv,sun,invb,sunb,temp.iloc[0].經度,temp.iloc[0].緯度])

        eliminate.append(combine)
  
    df = df[~(df['Date'].isin(duplicate_date))]

    df.reset_index(drop=True, inplace=True)

    for i in range(len(eliminate)):
        df.loc[len(df)+i+1]=eliminate[i]
  
    df = df.sort_values('Date')
    df = df.set_index(['Date']).asfreq('1D').ffill()

    if not dens_target_fill.empty:
        dens_target_fill = pd.concat([dens_target_fill,df])
    else:
        dens_target_fill = df
dens_target_fill

,Year,Month,Day,Week,區別,里別,調查戶數,陽性戶數,調查容器合計,陽性容器合計,經度,緯度
Date,,,,,,,,,,,,
2015-02-03,2015,2,3,05,永康區,北灣里,50.0,0.0,9.0,0.0,120.267333,23.111625
2015-02-04,2015,2,3,05,永康區,北灣里,50.0,0.0,9.0,0.0,120.267333,23.111625
2015-02-05,2015,2,3,05,永康區,北灣里,50.0,0.0,9.0,0.0,120.267333,23.111625
2015-02-06,2015,2,3,05,永康區,北灣里,50.0,0.0,9.0,0.0,120.267333,23.111625
2015-02-07,2015,2,3,05,永康區,北灣里,50.0,0.0,9.0,0.0,120.267333,23.111625
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-14,2020,9,14,37,安平區,億載里,50.0,3.0,10.0,3.0,120.161490,22.991994
2020-09-15,2020,9,14,37,安平區,億載里,50.0,3.0,10.0,3.0,120.161490,22.991994
2020-09-16,2020,9,14,37,安平區,億載里,50.0,3.0,10.0,3.0,120.161490,22.991994


In [21]:
dens_target_fill['布氏指數'] = dens_target_fill['陽性容器合計']/dens_target_fill['調查戶數']*100
dens_target_fill['容器指數'] = dens_target_fill['陽性容器合計']/dens_target_fill['調查容器合計']*100
dens_target_fill = dens_target_fill.reindex(columns = ['Year','Month','Day','Week','區別','里別','調查戶數','陽性戶數','調查容器合計','陽性容器合計','布氏指數','容器指數','經度','緯度'])

In [22]:
gap_count_2016 = []
dens_target_fill['Year'] = dens_target_fill['Year'].astype(int)

for i in range(len(target_area)):
    df = dens_target_fill[(dens_target_fill['區別']==target_area[i][0]) & (dens_target_fill['里別']==target_area[i][1])]
    df_2016 = df[df.Year>=2016]
    temp_gap_2016 = list(df_2016.groupby(['Year','Week']).count().index[df_2016.groupby(['Year','Week']).count().Day>=28])
    gap_count_2016.append(len(temp_gap_2016))

In [23]:
dens_final = pd.DataFrame()
for i in np.where(np.array(gap_count_2016)<10):
    for j in range(len(i)):
        df = dens_target_fill[(dens_target_fill['區別']==target_area[i[j]][0]) & (dens_target_fill['里別']==target_area[i[j]][1])]
        df_2016 = df[df.Year>=2016]
        if not dens_final.empty:
            dens_final = pd.concat([dens_final,df_2016])
        else:
            dens_final = df_2016
dens_final

,Year,Month,Day,Week,區別,里別,調查戶數,陽性戶數,調查容器合計,陽性容器合計,布氏指數,容器指數,經度,緯度
Date,,,,,,,,,,,,,,
2016-03-21,2016,3,21,12,永康區,北灣里,50.0,4.0,40.0,6.0,12.000000,15.00000,120.267333,23.111625
2016-03-22,2016,3,21,12,永康區,北灣里,50.0,4.0,40.0,6.0,12.000000,15.00000,120.267333,23.111625
2016-03-23,2016,3,21,12,永康區,北灣里,50.0,4.0,40.0,6.0,12.000000,15.00000,120.267333,23.111625
2016-03-24,2016,3,24,12,永康區,北灣里,53.0,3.0,62.0,3.0,5.660377,4.83871,120.267333,23.111625
2016-03-25,2016,3,24,12,永康區,北灣里,53.0,3.0,62.0,3.0,5.660377,4.83871,120.267333,23.111625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-14,2020,9,14,37,安平區,億載里,50.0,3.0,10.0,3.0,6.000000,30.00000,120.161490,22.991994
2020-09-15,2020,9,14,37,安平區,億載里,50.0,3.0,10.0,3.0,6.000000,30.00000,120.161490,22.991994
2020-09-16,2020,9,14,37,安平區,億載里,50.0,3.0,10.0,3.0,6.000000,30.00000,120.161490,22.991994


In [24]:
dens_final.to_csv('tainan_mosquito.csv',encoding='utf_8_sig')

In [35]:
cwd = os.getcwd()
path = os.path.join(cwd,'Temperature')
FileList = os.listdir(path)
temp = pd.DataFrame()
df = []
for file in FileList:
    df = pd.read_csv(os.path.join(path, file))
    if not temp.empty:
        temp = pd.concat([temp,df])
    else:
        temp = df
temp

,ObsTime,StnPres,SeaPres,StnPresMax,StnPresMaxTime,StnPresMin,StnPresMinTime,Temperature,T Max,T Max Time,...,PrecpHrMax,PrecpHrMaxTime,SunShine,SunShineRate,GloblRad,VisbMean,EvapA,UVI Max,UVI Max Time,Cloud Amount
0,1,1021,1024.2,1023,2016-01-01 09:07,1018.9,2016-01-01 14:27,18.7,24.4,2016-01-01 14:46,...,0.0,...,9.4,87.8,13.31,3.2,...,...,...,3.4
1,2,1018.9,1022.1,1021.1,2016-01-02 00:01,1016.4,2016-01-02 15:43,19.3,21.9,2016-01-02 11:08,...,1.5,2016-01-02 16:02,1.3,12.1,5.49,1.4,...,...,...,9.8
2,3,1016.6,1019.8,1018.7,2016-01-03 09:11,1014.7,2016-01-03 14:21,19.2,20.2,2016-01-03 15:33,...,19.5,2016-01-03 10:49,0,0,1.94,1.9,...,...,...,9.5
3,4,1015.6,1018.7,1017.8,2016-01-04 08:43,1013.9,2016-01-04 15:36,21.4,25.6,2016-01-04 14:10,...,0.0,...,1.4,13.1,8.08,1.6,...,...,...,6.9
4,5,1013.9,1017,1015.2,2016-01-05 00:01,1012,2016-01-05 15:34,23,26.8,2016-01-05 11:56,...,0.0,...,7.4,69,13.03,3.5,...,...,...,7.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,27,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,28,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,29,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,30,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [36]:
temp[['Date','Time']] = temp.StnPresMaxTime.str.split(' ',expand=True)
column = ['Date','Temperature']
temp = temp[column]
temp

,Date,Temperature
0,2016-01-01,18.7
1,2016-01-02,19.3
2,2016-01-03,19.2
3,2016-01-04,21.4
4,2016-01-05,23
...,...,...
26,...,...
27,...,...
28,...,...
29,...,...


In [37]:
idx = pd.date_range('2016-01-01', '2020-07-16')
idx[-1]-idx[0]

temp = temp[temp['Date'].str.contains('-', na=False)]
temp['Date'] = pd.to_datetime(temp['Date'])
temp = temp.sort_values('Date')

temp

C:\Users\yt335\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Date,Temperature
0,2016-01-01,18.7
1,2016-01-02,19.3
2,2016-01-03,19.2
3,2016-01-04,21.4
4,2016-01-05,23
...,...,...
11,2020-07-12,31.6
12,2020-07-13,31.3
13,2020-07-14,31.1
14,2020-07-15,29.3


In [38]:
idx = pd.date_range('2016-01-01', '2020-07-16')
temp = temp.set_index(['Date'])
temp = temp.reindex(idx).ffill()
temp = temp.reset_index()
temp = temp.rename(columns = {'index':'Date'})
temp

,Date,Temperature
0,2016-01-01,18.7
1,2016-01-02,19.3
2,2016-01-03,19.2
3,2016-01-04,21.4
4,2016-01-05,23
...,...,...
1654,2020-07-12,31.6
1655,2020-07-13,31.3
1656,2020-07-14,31.1
1657,2020-07-15,29.3


In [39]:
mos = pd.read_csv('tainan_mosquito.csv')
mos['Date'] = pd.to_datetime(mos['Date'])

mos.shape

(56327, 15)

In [40]:
data = pd.merge(mos,temp) 
data.shape

data = data.replace(np.nan,0)

data.isnull().any(axis = 0)

Date           False
Year           False
Month          False
Day            False
Week           False
區別             False
里別             False
調查戶數           False
陽性戶數           False
調查容器合計         False
陽性容器合計         False
布氏指數           False
容器指數           False
經度             False
緯度             False
Temperature    False
dtype: bool

In [41]:
data.to_csv('tainan_mosquito_temp.csv',encoding='utf_8_sig')